<a href="https://colab.research.google.com/github/KARENCMP82/Python/blob/main/1_METRICAS_DISTANCIAS_19MAR25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Nuclio Digital School - Máster en Data Science**

# Unsupervised Learning: Similitud y distancias

# *Profesora: Raquel Revilla*



<a id = "objetivos"></a>
# Objetivos del Notebook
[Volver al índice](#table_of_contents)

El presente notebook va a introducir al alumno a los conceptos básicos de **Aprendizaje No Supervisado (UL)**.

***¿Por qué decimos no supervisado?***

Tal y como su nombre indica, en el **Aprendizaje No Supervisado** tenemos un conjunto de datos de entrenamiento sin el correspondiente valor del target. <u>**Tenemos la X pero no la Y.**</u> Al principio podría parecer que si no tenemos el target de entrenamiento el dataset carece de valor, pero esto no es así.

Entre los principales problemas que buscan resolver los algoritmos de **Aprendizaje No Supervisado** se encuentran:

***¿Qué entendéis por clustering?***

1. **Clusterización/Segmentación de los datos**: se buscan grupos de <u>**ejemplos similares**</u> entre si y diferentes al resto de grupos. Por ejemplo: tenemos un dataset de clientes y queremos ver que grupos existen en función de determinados patrones que están ocultos en los datos. Ejecutando nuestro algoritmo de segmentación podríamos obtener las siguientes etiquetas: clientes digitales, clientes eco, clientes que compran exclusivamente en tiendas físicas, etc. Como es lógico, los clientes del grupo de "digitales" son muy parecidos entre sí y muy diferentes a los clientes "físicos".

***¿Qué es un recomendador?***

2. **Collaborative Filtering (Recommenders)**: con estos algoritmos queremos determinar <u>**la similitud**</u> entre ejemplos con el objetivo de ponderar alguna métrica para realizar recomendaciones. Se trata de ofrecer a clientes similares productos que vayan a gustar. Por ejemplo: tenemos un dataset de reviews de péliculas (n clientes que han hecho reviews a x péliculas). Ejecutando nuestro algoritmo de recomendación obtendriamos un score de similitud para cada cliente contra el resto de clientes. Sabríamos que un cliente se parece a otro porque han puesto notas similares a las mismas péliculas y con esta información podríamos construir una oferta personalizada de nuevas péliculas.


3. **Reducción de la dimensionalidad**: busca pasar de un espacio de **"m"** dimensiones a otro espacio de **"n"** dimensiones de tal manera que n << m. La utilidad de reducir la dimensionalidad de un dataset es para conseguir que sea mucho más manejable. Un dataset "pequeño" es más fácil de analizar, visualizar etc. Por este motivo, la reducción de la dimensionalidad suele ser un paso concreto dentro de un proyecto de Machine Learning (no un fin en si mismo).

![UL_INTRO](https://drive.google.com/uc?export=view&id=1CrALL3vdJxpY-MOWqLc-je92XjIJcyLj)

Un concepto recurrente que aparece en los puntos anteriores es **la idea de similitud.** Para agrupar a clientes similares (clústerización) o bien hacer recomendaciones basadas en gustos similares (collaborative filtering), **necesito poder calcular una métrica que me permita comparar a los clientes entre si de una manera objetiva.**

En este notebook veremos algunas de las formas más comunes para hacerlo.

Al final de la sesión, el alumno se debe sentir cómodo con los siguientes conceptos:

1. Comprender que es la distancia Euclídea y la de Manhattan (**CORE IDEA**).

2. **Aplicación de estas dos distancias a un dataset**, para encontrar en base a nuestras variables a los clientes más parecidos (los que más cerca están en nuestro espacio dimensional).

3. Ver en la práctica que problemas pueden surgir si tenemos diferentes escalas en nuestro dataset (**CORE IDEA**).

4. Aplicar la idea de distancias para ver algoritmos de "aprendizaje por instancias".

---

Las ideas más importantes están marcadas con **CORE IDEA** y recomendamos a los alumnos que centren sus esfuerzos en estos apartados.

Al final del notebook, hay un sección de referencias y lecturas recomendables para que el alumno pueda seguir profundizando en estos conceptos.

---

<a id = "table_of_contents"></a>

# Índice

[Importación de las Principales Librerías](#import_modules)

[GLOBAL_VARIABLES](#global_variables)

[Funciones Auxiliares](#helpers)

[Generación de un dataset](#dataset)

[Cálculo de distancias](#distances)

---> [Distancia Euclídea Sin Estandarización](#euclidea_bad)

---> [Distancia de Manhattan Sin Estandarización](#manhattan_bad)

---> [Distancia Euclídea Con Estandarización](#euclidea_bien)

---> [Distancia de Manhattan Con Estandarización](#manhattan_bien)

[Otras formas de procesar los datos](#transformers)

[Comparación de los Transformers](#scaler_comparison)

[Aprendizaje Basado en Instancias](#knn)

[Conclusión](#conclusión)

[Referencias y lecturas recomendables](#referencias)

<a id = "import_modules"></a>
# Importación de las Principales Librerías
[Volver al índice](#table_of_contents)

En esta sección hacemos los imports de las principales librerías.

In [ ]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import set_config
set_config(transform_output = "pandas")
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
from sklearn.datasets import load_diabetes
from sklearn.impute import KNNImputer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

<a id = "global_variables"></a>
# GLOBAL_VARIABLES
[Volver al índice](#table_of_contents)

Definimos las GLOBAL_VARIABLES que afectarán a nuestro notebook.

In [ ]:
COLOR = "#2a9d8f"
COLOR_MAX = "#D8E4FF"

<a id = "helpers"></a>
# Funciones Auxiliares
[Volver al índice](#table_of_contents)

Definimos las funciones que vamos a usar en el resto del notebook.

In [ ]:
def plot_euclidean_distance():
    '''
    Makes a simple plot of an Euclidean distances in a 2D space.
    Returns nothing. Renders the plot on the function call.
    '''
    # instanciamos la figure y el axes
    fig = plt.figure(figsize = (10, 10))
    ax = fig.add_subplot(111)

    # creamos una espacio cartesiona
    lims = (-10, 10)
    alpha = 0.5
    ax.set_xlim(lims)
    ax.set_ylim(lims)

    # pintamos lineas horizontales y verticales
    ax.vlines(0, -10, 10, linestyles = "--", alpha = alpha)
    ax.hlines(0, -10, 10, linestyles = "--", alpha = alpha)

    # pintamos los dos catetos y la hipotenusa
    ax.plot([0, 5], [0, 5], alpha = alpha, color = "red", lw = 2) # hipotenusa
    ax.text(2, 3.5, s = "d", fontsize = 8, color = "red")

    ax.plot([0, 5], [0, 0], alpha = alpha, color = "blue", lw = 2) # cateto azul
    ax.text(1.5, -1, s = "x2 - x1", fontsize = 8, color = "blue")

    ax.plot([5, 5], [5, 0], alpha = alpha, color = "green", lw = 2) # cateto verde

    # origen
    ax.scatter([0, 0], [0, 0], alpha = alpha, color = "black", lw = 2)
    ax.text(-1, -1, s = "(x1, y1)", fontsize = 8, color = "black")

    # punto 2
    ax.scatter([0, 5], [0, 5], alpha = alpha, color = "black", lw = 2)
    ax.text(5.5, 5.5, s = "(x2, y2)", fontsize = 8, color = "black")

    # punto en el eje x
    ax.scatter([5, 0], [0, 0], alpha = alpha, color = "black", lw = 2)

    # cálculo
    ax.text(1.5, -1, s = "x2 - x1", fontsize = 8, color = "blue")
    ax.text(5.5, 2, s = "y2 - y1", fontsize = 8, color = "green")

    # creamos las listas para los ejes (-10 hasta 10)
    x_ticks = [x for x in range(-10, 11)]
    y_ticks = [y for y in range(-10, 11)]

    # enumeramos los ejes desde -10 hasta 10
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)

    # ponemos el título
    fig.suptitle("Euclidean Distance Between 2 Vectors in a 2D Space");

In [ ]:
def calculate_distances(X, index, distance_func):
    '''
    Calculates the distances between vectors with the specified function you pass.
    Returns a pandas DataFrame.
    '''
    distances = distance_func(X = X)
    distances = pd.DataFrame(distances, index = index, columns = index)
    distances = round(distances, 1)
    distances.replace(to_replace = 0, value = np.nan, inplace = True)

    return distances

In [ ]:
def format_cell_based_on_target_value(value, target_value, highlight = 'background-color: yellow'):
    '''
    Formats a cell based on a target_value.
    Returns a background color or pass.
    '''
    if value == None:
        pass
    elif value == target_value:
        return highlight
    else:
        return ""

<a id = "dataset"></a>
# Generación de un dataset
[Volver al índice](#table_of_contents)

Vamos a generar un dataset dummy con el que vamos a trabajar.

In [ ]:
data = {
    "1. Edad":[18, 30, 21, 25, 33, 33],
    "2. Peso":[90, 70, 77, 60, 80, 80],
    "3. Altura":[180, 175, 170, 183, 185, 185],
    "4. Nómina":[2000, 1500, 3000, 1800, 900, 3000]
}

index = ["Cliente1", "Cliente2", "Cliente3", "Cliente4", "Cliente5", "Cliente6"]

Vamos a comenzar la sesión con una pregunta sencilla.

### Pregunta 1: ¿el Cliente6 a que cliente se le parece más?

In [ ]:
# NOTA IMPORTANTE:
# Hacemos el transpose para visualizar más fácil el dataset (cada cliente está en 1 columna)

# El alumno debe tener presente que
# el cálculo de las distancias se realizan a nivel de filas (cliente1 vs cliente2, cliente1 vs cliente3 etc...)

df_clientes = pd.DataFrame(data = data, index = index)
df_clientes.T

In [ ]:
df_clientes[["1. Edad"]].T

<a id = "distances"></a>
# Cálculo de distancias
[Volver al índice](#table_of_contents)

En el dataset anterior tenemos un conjunto de 6 clientes (6 instancias) y tengo recogidos un total de 4 variables para cada uno de ellos. Tengo representados a mis clientes en 4 dimensiones (Edad, Peso, Altura y Nómina).

Una idea muy sencilla e intuitiva para ver que clientes son más similares entre si es utilizar la noción de "proximidad". Por ejemplo: si sólo tuviera 1 dimensión (la edad), sería muy fácil concluir que Cliente6 y Cliente5 son los mas parecidos porque los dos tienen la misma edad. La distancia que les separa (en edad) es cero.

Veamos ahora como puedo generalizar esta idea a n dimensiones.

<a id = "euclidea_bad"></a>
# --> Similitud basada en la distancia Euclídea (dataset sin estandarizar)
[Volver al índice](#table_of_contents)

La [distancia Euclídea](https://es.wikipedia.org/wiki/Distancia_euclidiana) es la más utilizada dentro del mundo de **Aprendizaje No Supervisado**.

Desde un punto de vista matemático, es la distancia ordinaria entre dos puntos en un espacio euclídeo. Esto implica que el camino más corto entre 2 puntos es **"ir por la diagonal"**.

Se puede calcular fácilmente utilizando el teorema de Pitágoras (calculamos la hipotenusa a partir de los catetos de una triángulo).

Viene dada por la siguiente expresión matemática (para 2 dimensiones):

![Distancia_Euclídea](https://drive.google.com/uc?export=view&id=1-gQz_021Z_Dp_IDm2NIvhEu9ZdIAhvv4)

Se puede generalizar hasta N dimensiones con la siguiente fórmula:

![Distancia_Euclídea_General](https://drive.google.com/uc?export=view&id=1cbAXVW1c9CpZqMBcFe3nJTeVCJUKh4XH)

In [ ]:
plot_euclidean_distance()

Vamos a calcular la distancia euclídea para 2 clientes cualquiera y posteriormente lo haremos para todas las instancias de nuestro dataset.

In [ ]:
df_cl1_2 = df_clientes.T.iloc[:, :2]
df_cl1_2["Diff"] = df_cl1_2["Cliente1"] - df_cl1_2["Cliente2"]
df_cl1_2

In [ ]:
eucl_cl1_cl2 = (((18 - 30) ** 2) + ((90 - 70) ** 2) + ((180 - 175) ** 2) + ((2000 - 1500) ** 2)) ** 0.5
print(f"The Euclidean distance between Cliente1 and Cliente2 is {round(eucl_cl1_cl2, 1)}")

In [ ]:
df_clientes.T

In [ ]:
target_value = 500.6

In [ ]:
distances = calculate_distances(X = df_clientes, index = index, distance_func = euclidean_distances)

target_value = distances.iloc[1, 0]

distances.style.\
applymap(lambda cell_value: format_cell_based_on_target_value(value = cell_value, target_value = target_value)).\
highlight_min(axis = 1, color = COLOR)

<a id = "manhattan_bad"></a>
# --> Similitud basada en la distancia Manhattan (dataset sin estandarizar)
[Volver al índice](#table_of_contents)

La otra medida de distancia muy utilizada dentro del mundo del **Data Science** es la [distancia de Manhattan.](https://es.wikipedia.org/wiki/Geometr%C3%ADa_del_taxista)

Al contrario que la distancia Euclídea (donde nos desplazamos por la diagonal), en la distancia de Manhattan nos desplazamos por los catetos del triángulo (no podemos atravesar un edificio en el Eixample de Barcelona). Para ello tomamos el **valor absoluto** de la diferencia entre los elementos de cada vector.

La forma para calcular la distancia de Manhattan para N dimensiones viene dada por la siguiente fórmula:

![Distancia_Manhattan_General](https://drive.google.com/uc?export=view&id=1v6V3MmdsubMwubFpOmN_zk6LQ81mMqiR)


In [ ]:
manh_cl1_cl2 = (np.abs((18 - 30)) + np.abs((90 - 70)) + np.abs((180 - 175)) + np.abs((2000 - 1500)))
print(f"The Manhattan distance between Cliente1 and Cliente2 is {round(manh_cl1_cl2, 1)}")

In [ ]:
df_clientes.T

In [ ]:
distances = calculate_distances(X = df_clientes, index = index, distance_func = manhattan_distances)

target_value = distances.iloc[1, 0]

distances.style.\
applymap(lambda cell_value: format_cell_based_on_target_value(value = cell_value, target_value = target_value)).\
highlight_min(axis = 1, color = COLOR)

<a id = "euclidea_bien"></a>
# --> Similitud basada en la distancia Euclídea (dataset estandarizado)
[Volver al índice](#table_of_contents)

Hemos podido ver en los ejemplos anteriores que al tener las 4 variables diferentes escalas, la variable de nómina se lleva todo el "protagonismo".

La distancia entre el Cliente6 y Cliente4 es de 1.230 unidades. La diferencia de nómina que tienen es de 1.200.

Claramente esto no es lo que me interesa, porque quiero tener en cuenta las 4 dimensiones descriptivas de mis clientes para determinar la similitud.

Por este motivo, antes de calcular la distancia euclídea **tengo** que estandarizar mi dataset.

Vamos a volver a calcular las distancias entre clientes, estandarizando previamente mi dataset con el StandardScaler.

**RECORDAD:** StandardScaler no cambia la forma de la distribución, los valores escalados no están acotados pero sí que el resultado final es una dataset donde la media es cero y desviación típica es 1

In [ ]:
scaler = StandardScaler()

In [ ]:
scaled_df = scaler.fit_transform(df_clientes)
scaled_df

In [ ]:
distances = calculate_distances(
    X = scaled_df,
    index = index,
    distance_func = euclidean_distances
)

distances.style.highlight_min(axis = 1, color = COLOR)

<a id = "manhattan_bien"></a>
# --> Similitud basada en la distancia Manhattan (dataset estandarizado)
[Volver al índice](#table_of_contents)

In [ ]:
distances_scaled = calculate_distances(X = scaled_df, index = index, distance_func = manhattan_distances)
distances_scaled.style.highlight_min(axis = 1, color = COLOR)

<a id = "transformers"></a>
# --> Otras formas de procesar los datos
[Volver al índice](#table_of_contents)

Otra forma de estandarizar nuestro dataset es utilizar el MinMaxScaler.

**RECORDAD:** **MinMaxScaler** hace el valor más pequeño de cada columna sea 0 y el máximo 1, el resto de los valores están comprendidos entre este rango.

[Más sobre el MinMaxScaler de sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler)

In [ ]:
scaler = MinMaxScaler()

df_clientes_max_scaled = scaler.fit_transform(df_clientes)

df_clientes_max_scaled.columns = map(lambda name: "MMS" + name, df_clientes_max_scaled.columns)
df_clientes_max_scaled = pd.concat([df_clientes_max_scaled, df_clientes], axis = 1)
df_clientes_max_scaled

df_clientes_max_scaled.style.highlight_min(axis = 0, color = COLOR).highlight_max(axis = 0, color = COLOR_MAX)

<a id = "scaler_comparison"></a>
# Comparación de los Transformers
[Volver al índice](#table_of_contents)

En esta sección vamos a ver como afectan las diferentes transformaciones a la distribución y el rango de valores de un dataset.

Recordamos que tenemos varias maneras de transformar los datos:

*   ***StandardScaler :*** No cambia la forma de la distribución y los valores escalados no están acotados pero sí que el resultado final es una dataset donde la media es cero y desviación típica es 1
*   ***MinMaxScaler :*** Los valores están acotados entre 0 - 1 y no cambia la forma
*   ***Transformación logaritmica :*** En cambio aquí, sí que cambia la forma.  Esta transformación es particularmente útil cuando los datos tienen una distribución sesgada o presentan una gran variabilidad.
*   ***Robust Scaler :*** No tiene en cuenta a los outliers ya que utiliza la mediana y el rango intercuartílico para escalar los datos basándose en estadísticas robustas.

In [ ]:
X, y = load_diabetes(as_frame = True, return_X_y = True)

In [ ]:
y = y.to_frame()

In [ ]:
y

In [ ]:
y_standarized = pd.DataFrame(StandardScaler().fit_transform(y))
y_min_max = pd.DataFrame(MinMaxScaler().fit_transform(y))
y_log = pd.DataFrame(np.log(y))
y_robust_scaled = pd.DataFrame(RobustScaler(quantile_range = (10.0, 90.0)).fit_transform(y))

In [ ]:
KIND = "hist"

fig = plt.figure(figsize = (15, 5))
axes1, axes2 = fig.subplots(2, 3)

y.plot(kind = KIND, ax = axes1[0], title = "Original Data")
y_standarized.plot(kind = KIND, ax = axes1[1], title = "StandardScaler")
y_min_max.plot(kind = KIND, ax = axes1[2], title = "MixMaxScaler")
y_log.plot(kind = KIND, ax = axes2[0], title = "Log")
y_robust_scaled.plot(kind = KIND, ax = axes2[1], title = "RobustScaler")

<a id = "knn"></a>
# Aprendizaje Basado en Instancias
[Volver al índice](#table_of_contents)

Una vez que tenemos clara la idea de similitud y distancias, una cuestión muy interesante que puede surgir es:

**¿Podemos aprovechar la distancia euclídea para resolver problemas no supervisados?**

La respuesta es sí y uno de los algoritmos más usados para estas tareas se llama el [KNN (k-Nearest Neighbors)](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).

Este algoritmo se llama "lazy learners" porque no existe un entrenamiento tal y como succede con otros modelos como LogisticRegression, Redes Neuronales u otros.

En vez de esto, cada vez que me solicitan una predicción utilizo el KNN para "buscar" entre todas mis instancias y entre todas las dimensiones (variables) las ejemplos más próximos (utilizando la distancia euclídea) y hago mi predicción en función de la clase mayoritaría (problemas de clasificación) o bien voy a predecir la media en caso de problemas de regresión.

![KNN](https://drive.google.com/uc?export=view&id=1_lhK7u-MwqIU-cGg_caeu17HbUQL-WUX)

In [ ]:
# cargamos nuestro dataset
X_train = pd.read_csv("/content/drive/MyDrive/Nuclio_No_Supervisado/unsupervised_learning/data/pd_sklearn_data.csv")

# ponemos en el índice PassengerId
X_train.set_index("PassengerId", inplace = True)

# nos quedamos con las columnas numéricas
X_train = X_train[["Pclass", "Age", "SibSp", "Parch", "Fare", "Sex"]]

# transformamos columna Sex en booleana
X_train["Sex"] = X_train["Sex"].apply(lambda sex: 1 if sex == "female" else 0)

# creamos un indicador de que pasajero tenían nulos
X_train["Null_Age"] = X_train["Age"].isnull()

# guardamos el indicador de nulos en edad
null_age = X_train[["Null_Age"]]

# nos quedamos con las númericas
X_train.drop(columns = "Null_Age", inplace = True)

In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
# instanciamos nuestro KNNImputer
knn_imputer = KNNImputer(n_neighbors = 7)

In [ ]:
# Imputamos los nulos en base a 7 vecinos más cercanos
X_train_imputed = knn_imputer.fit_transform(X_train)

In [ ]:
# mergemos el flag de nulos y miramos que valores hemos imputado
X_train_imputed = X_train_imputed.merge(null_age, how = "left", left_index = True, right_index = True)

# filtramos algunos nulos a modo de ejemplos
X_train_imputed[X_train_imputed["Null_Age"] == True].head(10)

In [ ]:
X_train[X_train["Age"].isnull()].head(10)


El algoritmo de KNN es extremadamente potente y versátil, pero tiene 2 carencias fundamentales:
1. No es muy escalable porque para cada nueva predicción tengo que cargar toda la base de datos. Esto implica que para cada nueva predicción, tengo que calcular todas las distancias y ver cuales son estos vecinos más cercanos y en función de estos hacer el predict. Esto no succede con una Red Neuronal porque durante el entrenamiento aprendió del dataset y puede hacer el predict a partir de los paramétros inferidos en el train.


2. Un segundo problema que se presenta es que KNN necesita mucho espacio en memoria. Se tiene que cargar el dataset en memoria para hacer la búsqueda y en determinados contextos esto puede suponer un problema.

Por los dos motivos antes expuestos se recomienda usar el KNN para datasets pequeños.

<a id = "conclusión"></a>
# Conclusión
[Volver al índice](#table_of_contents)

Las principales conclusiones que podemos extraer de este notebook son:

1. La distancia Euclídea o la de Manhattan son dos formas de calcular la proximidad entre dos vectores en un espacio N dimensional. Esta distancia se puede utilizar en determinados contextos como "proxy" a similitud entre clientes. Algunos de los algoritmos que usan estas distancias son KMeans, KNN o DBScan.


2. Si nuestro dataset tiene variables que presentan diferentes escalas, es **fundamental** estandarizar antes estos valores (utilizando el StandardScaler o bien el MinMaxScaler) para que el aporte de cada atributo sea ~ el mismo.


3. Existen algoritmos que pueden hacer uso de la idea de "proximidad" para hacer muy buenas predicciones para datasets pequeños y medianos.

<a id='referencias'></a>
# Referencias y lecturas recomendables
[Volver al índice](#table_of_contents)<br>

A continuación dejamos algunos links útiles para profundizar en algunos de los conceptos que hemos visto en el notebook:

[Curse of Dimensionality](https://towardsdatascience.com/the-curse-of-dimensionality-50dc6e49aa1e)